In [4]:
#Ignoring TF warning messages
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" #

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from torchvision import models

In [6]:
# Define transformation for the CIFAR-10 images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the CIFAR-10 dataset
full_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

# Use only a small subset for demonstration (adjust as needed)
subset_size = 1000
subset_dataset, _ = random_split(full_dataset, [subset_size, len(full_dataset) - subset_size])

# DataLoader
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [7]:
# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the classifier to match the number of classes in CIFAR-10 (10 classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/yaduk/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [05:29<00:00, 1.68MB/s]  


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the training set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_accuracy = correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Accuracy: {train_accuracy * 100:.2f}%')

Epoch [1/5], Training Accuracy: 10.90%
Epoch [2/5], Training Accuracy: 10.90%
Epoch [3/5], Training Accuracy: 9.80%
Epoch [4/5], Training Accuracy: 10.90%
Epoch [5/5], Training Accuracy: 10.10%


In [9]:
# Print a message after training
print("Training complete.") #

Training complete.
